In [3]:
reset()
%display latex
Partitions.options.latex="list"
load("setup.sage")

KT_vars = ['x','y','z1','z2','z3','z4','z5','z6','z7','z8','z9',
           'v1','v2','v3','v4','v5','v6','v7','v8','v9']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)

coho_vars = ['t','m','l1','l2','l3','l4','l5','l6','l7','l8','l9',
             'w1','w2','w3','w4','w5','w6','w7','w8','w9']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)


x,y,z1,z2,z3,z4,z5,z6,z7,z8,z9,v1,v2,v3,v4,v5,v6,v7,v8,v9=KT_ring.gens()
t,m,l1,l2,l3,l4,l5,l6,l7,l8,l9,w1,w2,w3,w4,w5,w6,w7,w8,w9= coho_ring.gens()
b=var('b')
'''   w=SR.var('w',5)   '''
v=[v1,v2,v3,v4,v5,v6,v7,v8,v9]
w=[w1,w2,w3,w4,w5,w6,w7,w8,w9]
z=[z1,z2,z3,z4,z5,z6,z7,z8,z9]
L=var('l')


def character(p,x=x,y=y):
    return sum(y^(-r) * x^(-c) for r, c in p.cells())

def character_vb(p, w,x=x,y=y):
    Q = character(p,x,y)
    return sum(Q * x^(w[2*i]) * y^(w[2*i+1]) for i in range(len(w)/2))

def dual(f):
    R = f.parent()
    g= R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g

def tangent(p,x=x,y=y):
    total=0
    for r,c in p.cells():
        arm=p[r]-c-1
        leg=p.conjugate()[c]-r-1
        total+=y^(-leg)*x^(arm+1)+y^(leg+1)*x^(-arm)
    return total

def canonical_bundle(p,x=x,y=y):
    return character_vb(p, (-1,-1),x,y)


def structure_sheaf(p,x=x,y=y):
    return character_vb(p, (0,0),x,y)

def top_chern(f):
    numer, denom = [], []
    for coeff, mon in list(f):
        wt = from_monomial(mon)
        #if wt==0:
            #continue
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    
    return coho_ring.fraction_field()(prod(numer) / prod(denom))

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def from_monomial(m):
    return sum(m.degree(KT_ring.gen(i)) * coho_ring.gen(i) for i in range(len(coho_ring.gens())))

def from_character(f):
    numer, denom = [], []

    for coeff, mon in list(f):
        wt = from_monomial(mon)
        if coeff > 0:
            numer += (coeff) * [wt]
        elif coeff < 0:
            denom += (-coeff) * [wt]
    return numer, denom

def measure_unsymmetrized(f, inv=True):
    """
    computes 1/(1-x^-1) for K-thy class x. If inv=false, returns 1/(1-x)
    """
    R = f.parent()
    
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)

    numer, denom = R.one(), R.one()
    for coeff, monomial in L:
        if inv:
            term = 1 - monomial^-1
        else:
            term = 1 - monomial
        if monomial==1:
            term = 0
        if coeff < 0:
            numer *= term ** (-coeff)
        elif coeff > 0:
            denom *= term ** coeff
    return numer / denom



def degn(f,n,b=b):
    g=f.taylor(b,0,n+2)
    L=g.coefficients(b)
    for i in range(len(L)):
        if L[i][1]==n:
            return L[i][0]
    return 0

    

In [45]:
def partitions(Sum, K):
    '''computes fixed components of Quot by CC^* action.'''
    total=[]
    def ptn(Sum, K, lst, total):
        if K == 0:
            if Sum == 0:
                total+=[lst]
            return
        for i in range(0, min(Sum + 1, Sum + 1)):
            ptn(Sum - i, K - 1, lst + [i],total)
    ptn(Sum,K,[],total)
    return total


def fixed_locus(n, N):
    '''gives list of fixed points of Hilb^n1 x Hilb^n2 x ... x Hilb^nN, input (n1,n2,...,nN)'''
    total=[]
    ptns=partitions(n,N)
    for j in range(len(ptns)):
        ptn=ptns[j]
        lst=[]
        for i in range(N):
            lst+=[Partitions(ptn[i]).list()]
        total+=[p for p in itertools.product(*lst)]
    return total

def Nvir(Z):
    '''virtual normal bundle of quot scheme at (Z_1,...,Z_N)'''
    total=0
    for i in range(len(Z)):
        for j in range(len(Z)):
            total+=(1-character(Z[i],x=x^-1,y=x)*(1-x)*(1-1/x))*character(Z[j],x=x,y=x^-1)*(z[j]/z[i])
    return total+1

def total_chern_quot(Z,rank):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((1+r)(t=t,
                         l1=0,l2=0,l3=0,l4=0,l5=0,
                         w1=w1,w2=0,w3=0,w4=0,w5=0,w6=0,w7=0,w8=0,w9=0) for r in roots)
    return total

def top_chern_quot(Z):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(z[i])
        roots = []
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod((r) for r in roots)
    return total

def Chern_Num_quot(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=(top_chern(-Nvir(Z))*restrict(total_chern_quot(Z,rank),n+1))
        total+=add(w1=w1*t)
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def DT_quot(n,N):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))
        total+=add
    return total

def total_segre_quot(Z,rank):
    total=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(z[i])
        roots = []
    
        for coeff, mon in list(Q):
            wt = from_monomial(mon)
            for i in range(rank):
                roots += (coeff) * [wt+w[i]]
        total *= prod(1/(1+r) for r in roots)
    return total

def Segre_Num_quot(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_segre_quot(Z,rank)
        total+=add(t=7,l1=15,l2=2,l3=3,w1=17,w2=5,w3=6)
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def Segre_Num_quot_sym(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*total_segre_quot(Z,rank)
        total+=add(t=7,w1=15,w2=2,w3=3,l1=17,l2=5,l3=6)
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total


def determinant_quot(Z,n,rank):
    det=1
    for i in range(len(Z)):
        Q = character(Z[i],x,1/x)*(z[i])
        det *= determinant(Q)
    return (det^rank)*((prod(v[i] for i in range(rank)))^n)

def Verlinde_Num_quot(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            determinant_quot(Z,n,rank)
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def Verlinde_Num_quot_inv(n,N,rank):
    total=0
    maxcounter=len(fixed_locus(n,N))
    counter=0
    for Z in fixed_locus(n,N):
        add=measure_unsymmetrized(Nvir(Z))*\
            (1/determinant_quot(Z,n,rank))
        total+=add
        counter+=1
        print("added"+str(counter)+"/"+str(maxcounter))
    return total

def todd_class_fast(f,n):
    R = f.parent()
    total=1
    if f.parent() is ZZ:
        L = [(ZZ(f), 1)]
    else:
        L = list(f)
    for coeff, mon in L:
        t=1
        if mon==1:
            term = 0
        if coeff < 0:
            term = 1+sum((-1)^(k+1)*(from_monomial(mon))^(k+1)/factorial(k+2) for k in range(n))
            for i in range(-coeff):
                t=cut_off_n(t*term,n)
        elif coeff > 0:
            term = 1+sum(bernoulli_polynomial(1, k+1)*(from_monomial(mon))^(k+1)/factorial(k+1) for k in range(n))
            for i in range(coeff):
                t=cut_off_n(t*term,n)
        total=cut_off_n(total*t,n)
    return total

def cut_off_n(f,n):
    if f in ZZ:
        return f
    D=f.dict()
    for exp,coeff in f.dict().items():
        if sum(exp)>n:
            del D[exp]
    return coho_ring(D)

def restrict(f,n):
    D=f.dict()
    for exp,coeff in f.dict().items():
        if sum(exp)!=n:
            del D[exp]
    return coho_ring(D)

def specialize(f):
    if f in ZZ:
        return f
    return f(y=x^-1)

def determinant_fast(f,n):
    if f in ZZ:
        return 1
    else:
        total=1
        for coeff,mon in f:
            term=1+sum((coeff*from_monomial(mon))^(k+1)/factorial(k+1) for k in range(n+1))
            total=cut_off_n(total*term,n+1)
        return total

def determinant_quot_fast(Z,n, rank,N):
    det=1
    v_1=1
    v_2=1
    Q_2=1
    for i in range(rank):
        v_1=cut_off_n(v_1*(1+sum((w[i])^(k+1)/factorial(k+1) for k in range(n*N+1))),n*N+1)
    for i in range(n):
        v_2=cut_off_n(v_2*v_1,n*N+1)
    for i in range(len(Z)):
        Q = specialize(character(Z[i]))*(z[i])
        det = cut_off_n(det* determinant_fast(Q,n*N+1),n*N+1)
    for i in range(rank):
        Q_2=cut_off_n(Q_2*det,n*N+1)
    return cut_off_n(Q_2*v_2,n*N+1)

def determinant_fast_inv(f,n):
    if f in ZZ:
        return 1
    else:
        total=1
        for coeff,mon in f:
            term=1+sum((-coeff*from_monomial(mon))^(k+1)/factorial(k+1) for k in range(n+1))
            total=cut_off_n(total*term,n+1)
        return total

def determinant_quot_fast_inv(Z,n, rank,N):
    det=1
    v_1=1
    v_2=1
    Q_2=1
    for i in range(rank):
        v_1=cut_off_n(v_1*(1+sum((-w[i])^(k+1)/factorial(k+1) for k in range(n*N+1))),n*N+1)
    for i in range(n):
        v_2=cut_off_n(v_2*v_1,n*N+1)
    for i in range(len(Z)):
        Q = specialize(character(Z[i]))*(z[i])
        det = cut_off_n(det* determinant_fast_inv(Q,n*N+1),n*N+1)
    for i in range(rank):
        Q_2=cut_off_n(Q_2*det,n*N+1)
    return cut_off_n(Q_2*v_2,n*N+1)

def Verlinde_Num_quot_fast(n, r):
    total=0
    for Z in fixed_locus(n,1):
        add=top_chern(-Nvir(Z))*\
            restrict(todd_class_fast(Nvir(Z),n+1)*determinant_quot_fast(Z,n,r,1),n+1)
        total+=add
    return total(l1=0,w1=t*w1,w2=0,w3=0,w4=0,w5=0,w6=0,w7=0,w8=0,w9=0)

def Verlinde_Num_quot_fast_inv(n,N, r):
    total=0
    for Z in fixed_locus(n,N):
        add=top_chern(-Nvir(Z))*\
            restrict(todd_class_fast(Nvir(Z),n*N+1)*determinant_quot_fast_inv(Z,n,r,N),n*N+1)
        total+=add
    return total(l1=0,l2=0,l3=0,l4=0,l5=0,w1=t*w1,w2=0,w3=0,w4=0,w5=0,w6=0,w7=0,w8=0,w9=0)


def deg0(f):
    if f==0: 
        return 0
    return degn(f(t=b*t,
                         l1=0,l2=0,l3=0,l4=0,l5=0,
                         w1=b*t*w1,w2=0,w3=0,w4=0,w5=0,w6=0,w7=0,w8=0,w9=0),0)

def deg1(f):
    if f==0: 
        return 0
    return degn(f(t=b*t,m=b*m,
                         l1=b*l1,l2=b*l2,l3=b*l3,l4=b*l4,l5=b*l5,
                         w1=b*w1,w2=b*w2,w3=b*w3,w4=b*w4,w5=b*w5),-1)

def deg02(f):
    if f==0: 
        return 0
    return pretty(degn(f(t=b*t,m=b*m,w=b*w,v=b*v),0))

def set_exp(f):
    for i in range(len(KT_vars)):
        f=SR(f).subs(KT_ring.gens()[i]==exp(coho_ring.gens()[i]))
    return f
def simplify_weight(f):
    return f(l1=0,l2=l1,l3=2*l1,l4=3*l1,l5=4*l5)

def symmetrize(f):
    if f in ZZ:
        return f
    return f(l1=w1,l2=w2,l3=w3,l4=w4,l5=w5,w1=l1,w2=l2,w3=l3,w4=l4,w5=l5)


def symmetrizeK(f):
    if f in ZZ:
        return f
    return f(z1=v1,z2=v2,z3=v3,z4=v4,z5=v5,v1=z1,v2=z2,v3=z3,v4=z4,v5=z5)

def all_check(start,finish):
    with open("V2d-red-checked.txt") as f:
        lines = f.read().splitlines()
    if len(lines)==0:
        n,N,r=[0,0,0]
    else:
        last_checked=lines[len(lines)-1]
        n,N,r=[int(a) for a in last_checked.split(",")]
    for i in range(start,finish+1):
        for j in range(1,4):
            for k in range(1,4):
                n,N,r=[i,j,k]
                print("checking n="+str(n)+", N="+str(N)+", r="+str(r))
                if str(n)+','+str(N)+','+str(r) in lines:
                    print("n="+str(n)+", N="+str(N)+", r="+str(r)+" already checked")
                else:
                    a=ver_compare(n,N,r)
                    V=deg0(set_exp(Verlinde_Num_quot(n,N,r))).coefficient(w1^2)
                    diff=a-V
                    if diff==0:
                        writeline("V2d-red-checked.txt",str(n)+','+str(N)+','+str(r))
                        print("checked")
                    else:
                        print("failed")
        print("all checked for: n="+str(i))
                        
                    
def writeline(file,s):
    g = open(file, "a")
    g.write(s+"\n")
    g.close()
                
                
def clear_all_files():
    open('V2d-red-checked.txt', 'w').close()
            
        
def binom(n,k):
    return prod((n-i) for i in range(k))/prod((k-i) for i in range(k))
    
    
def ver_compare(n,N,r):
    return -N/(2*(N+r)) *  binom((N+r)*n,r*n)

def ver10_compare(n,r):
    return ((n-1) *  binom(r+1,2)-1)/6

def c10_compare(n,r):
    return ((r-1)^2 +(n-2)*binom(r,2)-1)/6

def check01(start,finish):
    # check for n,N,r, up to r=5
    with open("V2d01-red-checked.txt") as f:
        lines = f.read().splitlines()
    for i in range(start,finish+1):
        for k in range(1,5):
            n,r=[i,k]
            print("checking n="+str(n)+", r="+str(r))
            if str(n)+','+str(r) in lines:
                print("n="+str(n)+", r="+str(r)+" already checked")
            else:
                a=ver10_compare(n,r)
                X=deg0(set_exp(Verlinde_Num_quot(n,1,r)))
                V1=X.coefficient(w1^2)
                V0=X(w1=0)
                diff=a-V0/V1
                if V1==0:
                    if V0==0:
                        writeline("V2d01-red-checked.txt",str(n)+','+str(r))
                        print("checked")
                        continue
                    else:
                        print("failed")
                        print(a)
                        print(V0)
                        print(V1)
                diff=a-V0/V1
                if diff==0:
                    writeline("V2d01-red-checked.txt",str(n)+','+str(r))
                    print("checked")
                else:
                    print("failed")
                    print(a)
                    print(V0)
                    print(V1)
        print("all checked for: n="+str(i))
        
def check01_inv(start,finish):
    with open("V2d01-red-checked.txt") as f:
        lines = f.read().splitlines()
    for i in range(start,finish+1):
        for k in range(1,5):
            n,r=[i,k]
            print("checking n="+str(n)+", r="+str(-r))
            if str(n)+','+str(-r) in lines:
                print("n="+str(n)+", r="+str(-r)+" already checked")
            else:
                a=ver10_compare(n,-r)
                X=deg0(set_exp(Verlinde_Num_quot_inv(n,1,r)))
                V1=X.coefficient(w1^2)
                V0=X(w1=0)
                if V1==0:
                    if V0==0:
                        writeline("V2d01-red-checked.txt",str(n)+','+str(-r))
                        print("checked")
                        continue
                    else:
                        print("failed")
                        print(a)
                        print(V0)
                        print(V1)
                diff=a-V0/V1
                if diff==0:
                    writeline("V2d01-red-checked.txt",str(n)+','+str(-r))
                    print("checked")
                else:
                    print("failed")
                    print(a)
                    print(V0)
                    print(V1)
        print("all checked for: n="+str(i))
        
def binom(n,k):
    if n==0 and k==0:
        return 1
    if n==0:
        return 0
    if n>0 and k<0:
        return 0
    if k>=0:
        return ((1+b)^n).taylor(b,0,k).list()[k]
    else:
        return ((1+b^-1)^n).taylor(b,0,-k).list()[-1*k]
    
def all_check_inv(start,finish):
    with open("V2d-red-checked_inv.txt") as f:
        lines = f.read().splitlines()
    for i in range(start,finish+1):
        for j in range(1,4):
            for k in range(1,4):
                n,N,r=[i,j,k]
                if r==N: 
                    continue
                print("checking n="+str(n)+", N="+str(N)+", r=-"+str(r))
                if str(n)+','+str(N)+',-'+str(r) in lines:
                    print("n="+str(n)+", N="+str(N)+", r=-"+str(r)+" already checked")
                else:
                    a=ver_compare(n,N,-r)
                    VV=deg0(set_exp(Verlinde_Num_quot_inv(n,N,r)))
                    if VV==0:
                        V=0
                    else:
                        V=VV.coefficient(w1^2)
                    diff=a-V
                    if diff==0:
                        writeline("V2d-red-checked_inv.txt",str(n)+','+str(N)+',-'+str(r))
                        print("checked")
                    else:
                        print("failed")
        print("all checked for: n="+str(i))
        
def check01_seg(start,finish):
    with open("s2d-checked.txt") as f:
        lines = f.read().splitlines()
    for i in range(start,finish+1):
        for k in range(2,5):
            n,r=[i,k]
            print("checking n="+str(n)+", r="+str(r))
            if str(n)+','+str(r) in lines:
                print("n="+str(n)+", r="+str(r)+" already checked")
            else:
                a=c10_compare(n,r)
                X=Chern_Num_quot(n,1,r)
                V1=X.coefficient(w1^2)
                V0=X(w1=0)
                diff=a-V0/V1
                if V1==0:
                    if V0==0:
                        writeline("s2d-checked.txt",str(n)+','+str(r))
                        print("checked")
                        continue
                    else:
                        print("faile1")
                        print(a)
                        print(V0)
                        print(V1)
                        break
                diff=a-V0/V1
                if diff==0:
                    writeline("s2d-checked.txt",str(n)+','+str(r))
                    print("checked")
                else:
                    print("failed")
                    print(a)
                    print(V0)
                    print(V1)
                    break

In [46]:
Chern_Num_quot(8,1,4)

added1/22
added2/22
added3/22
added4/22
added5/22
added6/22
added7/22
added8/22
added9/22
added10/22
added11/22
added12/22
added13/22
added14/22
added15/22
added16/22
added17/22
added18/22
added19/22
added20/22
added21/22
added22/22


-1781325/2*w1^2 - 6531525

In [43]:
c10_compare(3,3)

1

In [47]:
check01_seg(8,20)

checking n=8, r=2
n=8, r=2 already checked
checking n=8, r=3
n=8, r=3 already checked
checking n=8, r=4
added1/22
added2/22
added3/22
added4/22
added5/22
added6/22
added7/22
added8/22
added9/22
added10/22
added11/22
added12/22
added13/22
added14/22
added15/22
added16/22
added17/22
added18/22
added19/22
added20/22
added21/22
added22/22
checked
checking n=9, r=2
added1/30
added2/30
added3/30
added4/30
added5/30
added6/30
added7/30
added8/30
added9/30
added10/30
added11/30
added12/30
added13/30
added14/30
added15/30
added16/30
added17/30
added18/30
added19/30
added20/30
added21/30
added22/30
added23/30
added24/30
added25/30
added26/30
added27/30
added28/30
added29/30
added30/30
checked
checking n=9, r=3
added1/30
added2/30
added3/30
added4/30
added5/30
added6/30
added7/30
added8/30
added9/30
added10/30
added11/30
added12/30
added13/30
added14/30
added15/30
added16/30
added17/30
added18/30
added19/30
added20/30
added21/30
added22/30
added23/30
added24/30
added25/30
added26/30
added27/30
ad

added75/101
added76/101
added77/101
added78/101
added79/101
added80/101
added81/101
added82/101
added83/101
added84/101
added85/101
added86/101
added87/101
added88/101
added89/101
added90/101
added91/101
added92/101
added93/101
added94/101
added95/101
added96/101
added97/101
added98/101
added99/101
added100/101
added101/101
checked
checking n=13, r=3
added1/101
added2/101
added3/101
added4/101
added5/101
added6/101
added7/101
added8/101
added9/101
added10/101
added11/101
added12/101
added13/101
added14/101
added15/101
added16/101
added17/101
added18/101
added19/101
added20/101
added21/101
added22/101
added23/101
added24/101
added25/101
added26/101
added27/101
added28/101
added29/101
added30/101
added31/101
added32/101
added33/101
added34/101
added35/101
added36/101
added37/101
added38/101
added39/101
added40/101
added41/101
added42/101
added43/101
added44/101
added45/101
added46/101
added47/101
added48/101
added49/101
added50/101
added51/101
added52/101
added53/101
added54/101
added55/

added37/176
added38/176
added39/176
added40/176
added41/176
added42/176
added43/176
added44/176
added45/176
added46/176
added47/176
added48/176
added49/176
added50/176
added51/176
added52/176
added53/176
added54/176
added55/176
added56/176
added57/176
added58/176
added59/176
added60/176
added61/176
added62/176
added63/176
added64/176
added65/176
added66/176
added67/176
added68/176
added69/176
added70/176
added71/176
added72/176
added73/176
added74/176
added75/176
added76/176
added77/176
added78/176
added79/176
added80/176
added81/176
added82/176
added83/176
added84/176
added85/176
added86/176
added87/176
added88/176
added89/176
added90/176
added91/176
added92/176
added93/176
added94/176
added95/176
added96/176
added97/176
added98/176
added99/176
added100/176
added101/176
added102/176
added103/176
added104/176
added105/176
added106/176
added107/176
added108/176
added109/176
added110/176
added111/176
added112/176
added113/176
added114/176
added115/176
added116/176
added117/176
added118/1

added169/231
added170/231
added171/231
added172/231
added173/231
added174/231
added175/231
added176/231
added177/231
added178/231
added179/231
added180/231
added181/231
added182/231
added183/231
added184/231
added185/231
added186/231
added187/231
added188/231
added189/231
added190/231
added191/231
added192/231
added193/231
added194/231
added195/231
added196/231
added197/231
added198/231
added199/231
added200/231
added201/231
added202/231
added203/231
added204/231
added205/231
added206/231
added207/231
added208/231
added209/231
added210/231
added211/231
added212/231
added213/231
added214/231
added215/231
added216/231
added217/231
added218/231
added219/231
added220/231
added221/231
added222/231
added223/231
added224/231
added225/231
added226/231
added227/231
added228/231
added229/231
added230/231
added231/231
checked
checking n=16, r=3
added1/231
added2/231
added3/231
added4/231
added5/231
added6/231
added7/231
added8/231
added9/231
added10/231
added11/231
added12/231
added13/231
added14

added128/297
added129/297
added130/297
added131/297
added132/297
added133/297
added134/297
added135/297
added136/297
added137/297
added138/297
added139/297
added140/297
added141/297
added142/297
added143/297
added144/297
added145/297
added146/297
added147/297
added148/297
added149/297
added150/297
added151/297
added152/297
added153/297
added154/297
added155/297
added156/297
added157/297
added158/297
added159/297
added160/297
added161/297
added162/297
added163/297
added164/297
added165/297
added166/297
added167/297
added168/297
added169/297
added170/297
added171/297
added172/297
added173/297
added174/297
added175/297
added176/297
added177/297
added178/297
added179/297
added180/297
added181/297
added182/297
added183/297
added184/297
added185/297
added186/297
added187/297
added188/297
added189/297
added190/297
added191/297
added192/297
added193/297
added194/297
added195/297
added196/297
added197/297
added198/297
added199/297
added200/297
added201/297
added202/297
added203/297
added204/297

...

added168/490
added169/490
added170/490
added171/490
added172/490
added173/490
added174/490
added175/490
added176/490
added177/490
added178/490
added179/490
added180/490
added181/490
added182/490
added183/490
added184/490
added185/490
added186/490
added187/490
added188/490
added189/490
added190/490
added191/490
added192/490
added193/490
added194/490
added195/490
added196/490
added197/490
added198/490
added199/490
added200/490
added201/490
added202/490
added203/490
added204/490
added205/490
added206/490
added207/490
added208/490
added209/490
added210/490
added211/490
added212/490
added213/490
added214/490
added215/490
added216/490
added217/490
added218/490
added219/490
added220/490
added221/490
added222/490
added223/490
added224/490
added225/490
added226/490
added227/490
added228/490
added229/490
added230/490
added231/490
added232/490
added233/490
added234/490
added235/490
added236/490
added237/490
added238/490
added239/490
added240/490
added241/490
added242/490
added243/490
added244/490

added315/490
added316/490
added317/490
added318/490
added319/490
added320/490
added321/490
added322/490
added323/490
added324/490
added325/490
added326/490
added327/490
added328/490
added329/490
added330/490
added331/490
added332/490
added333/490
added334/490
added335/490
added336/490
added337/490
added338/490
added339/490
added340/490
added341/490
added342/490
added343/490
added344/490
added345/490
added346/490
added347/490
added348/490
added349/490
added350/490
added351/490
added352/490
added353/490
added354/490
added355/490
added356/490
added357/490
added358/490
added359/490
added360/490
added361/490
added362/490
added363/490
added364/490
added365/490
added366/490
added367/490
added368/490
added369/490
added370/490
added371/490
added372/490
added373/490
added374/490
added375/490
added376/490
added377/490
added378/490
added379/490
added380/490
added381/490
added382/490
added383/490
added384/490
added385/490
added386/490
added387/490
added388/490
added389/490
added390/490
added391/490

added466/627
added467/627
added468/627
added469/627
added470/627
added471/627
added472/627
added473/627
added474/627
added475/627
added476/627
added477/627
added478/627
added479/627
added480/627
added481/627
added482/627
added483/627
added484/627
added485/627
added486/627
added487/627
added488/627
added489/627
added490/627
added491/627
added492/627
added493/627
added494/627
added495/627
added496/627
added497/627
added498/627
added499/627
added500/627
added501/627
added502/627
added503/627
added504/627
added505/627
added506/627
added507/627
added508/627
added509/627
added510/627
added511/627
added512/627
added513/627
added514/627
added515/627
added516/627
added517/627
added518/627
added519/627
added520/627
added521/627
added522/627
added523/627
added524/627
added525/627
added526/627
added527/627
added528/627
added529/627
added530/627
added531/627
added532/627
added533/627
added534/627
added535/627
added536/627
added537/627
added538/627
added539/627
added540/627
added541/627
added542/627

added480/627
added481/627
added482/627
added483/627
added484/627
added485/627
added486/627
added487/627
added488/627
added489/627
added490/627
added491/627
added492/627
added493/627
added494/627
added495/627
added496/627
added497/627
added498/627
added499/627
added500/627
added501/627
added502/627
added503/627
added504/627
added505/627
added506/627
added507/627
added508/627
added509/627
added510/627
added511/627
added512/627
added513/627
added514/627
added515/627
added516/627
added517/627
added518/627
added519/627
added520/627
added521/627
added522/627
added523/627
added524/627
added525/627
added526/627
added527/627
added528/627
added529/627
added530/627
added531/627
added532/627
added533/627
added534/627
added535/627
added536/627
added537/627
added538/627
added539/627
added540/627
added541/627
added542/627
added543/627
added544/627
added545/627
added546/627
added547/627
added548/627
added549/627
added550/627
added551/627
added552/627
added553/627
added554/627
added555/627
added556/627

added490/627
added491/627
added492/627
added493/627
added494/627
added495/627
added496/627
added497/627
added498/627
added499/627
added500/627
added501/627
added502/627
added503/627
added504/627
added505/627
added506/627
added507/627
added508/627
added509/627
added510/627
added511/627
added512/627
added513/627
added514/627
added515/627
added516/627
added517/627
added518/627
added519/627
added520/627
added521/627
added522/627
added523/627
added524/627
added525/627
added526/627
added527/627
added528/627
added529/627
added530/627
added531/627
added532/627
added533/627
added534/627
added535/627
added536/627
added537/627
added538/627
added539/627
added540/627
added541/627
added542/627
added543/627
added544/627
added545/627
added546/627
added547/627
added548/627
added549/627
added550/627
added551/627
added552/627
added553/627
added554/627
added555/627
added556/627
added557/627
added558/627
added559/627
added560/627
added561/627
added562/627
added563/627
added564/627
added565/627
added566/627

In [180]:
all_check_inv(1,9)

checking n=1, N=1, r=-2
n=1, N=1, r=-2 already checked
checking n=1, N=1, r=-3
n=1, N=1, r=-3 already checked
checking n=1, N=2, r=-1
n=1, N=2, r=-1 already checked
checking n=1, N=2, r=-3
n=1, N=2, r=-3 already checked
checking n=1, N=3, r=-1
n=1, N=3, r=-1 already checked
checking n=1, N=3, r=-2
n=1, N=3, r=-2 already checked
all checked for: n=1
checking n=2, N=1, r=-2
n=2, N=1, r=-2 already checked
checking n=2, N=1, r=-3
n=2, N=1, r=-3 already checked
checking n=2, N=2, r=-1
n=2, N=2, r=-1 already checked
checking n=2, N=2, r=-3
n=2, N=2, r=-3 already checked
checking n=2, N=3, r=-1
n=2, N=3, r=-1 already checked
checking n=2, N=3, r=-2
n=2, N=3, r=-2 already checked
all checked for: n=2
checking n=3, N=1, r=-2
n=3, N=1, r=-2 already checked
checking n=3, N=1, r=-3
n=3, N=1, r=-3 already checked
checking n=3, N=2, r=-1
n=3, N=2, r=-1 already checked
checking n=3, N=2, r=-3
n=3, N=2, r=-3 already checked
checking n=3, N=3, r=-1
n=3, N=3, r=-1 already checked
checking n=3, N=3, r=-2

In [223]:
check01(5,5)
check01_inv(5,5)

checking n=5, r=1
added1/7
added2/7
added3/7
added4/7
added5/7
added6/7
added7/7
checked
checking n=5, r=2
added1/7
added2/7
added3/7
added4/7
added5/7
added6/7
added7/7
checked
checking n=5, r=3
added1/7
added2/7
added3/7
added4/7
added5/7
added6/7
added7/7
checked
all checked for: n=5
checking n=5, r=-1
n=5, r=-1 already checked
checking n=5, r=-2
n=5, r=-2 already checked
checking n=5, r=-3
n=5, r=-3 already checked
all checked for: n=5


In [220]:
check01_inv(1,4)

checking n=1, r=-1
added1/1
checked
checking n=1, r=-2
n=1, r=-2 already checked
checking n=1, r=-3
n=1, r=-3 already checked
all checked for: n=1
checking n=2, r=-1
added1/2
added2/2
checked
checking n=2, r=-2
n=2, r=-2 already checked
checking n=2, r=-3
n=2, r=-3 already checked
all checked for: n=2
checking n=3, r=-1
added1/3
added2/3
added3/3
checked
checking n=3, r=-2
n=3, r=-2 already checked
checking n=3, r=-3
n=3, r=-3 already checked
all checked for: n=3
checking n=4, r=-1
added1/5
added2/5
added3/5
added4/5
added5/5
checked
checking n=4, r=-2
n=4, r=-2 already checked
checking n=4, r=-3
n=4, r=-3 already checked
all checked for: n=4


In [48]:
for n in range(1,21):
    check01(n,n)
    check01_inv(n,n)

checking n=1, r=1
n=1, r=1 already checked
checking n=1, r=2
n=1, r=2 already checked
checking n=1, r=3
n=1, r=3 already checked
all checked for: n=1
checking n=1, r=-1
n=1, r=-1 already checked
checking n=1, r=-2
n=1, r=-2 already checked
checking n=1, r=-3
n=1, r=-3 already checked
all checked for: n=1
checking n=2, r=1
n=2, r=1 already checked
checking n=2, r=2
n=2, r=2 already checked
checking n=2, r=3
n=2, r=3 already checked
all checked for: n=2
checking n=2, r=-1
n=2, r=-1 already checked
checking n=2, r=-2
n=2, r=-2 already checked
checking n=2, r=-3
n=2, r=-3 already checked
all checked for: n=2
checking n=3, r=1
n=3, r=1 already checked
checking n=3, r=2
n=3, r=2 already checked
checking n=3, r=3
n=3, r=3 already checked
all checked for: n=3
checking n=3, r=-1
n=3, r=-1 already checked
checking n=3, r=-2
n=3, r=-2 already checked
checking n=3, r=-3
n=3, r=-3 already checked
all checked for: n=3
checking n=4, r=1
n=4, r=1 already checked
checking n=4, r=2
n=4, r=2 already chec

added225/627
added226/627
added227/627
added228/627
added229/627
added230/627
added231/627
added232/627
added233/627
added234/627
added235/627
added236/627
added237/627
added238/627
added239/627
added240/627
added241/627
added242/627
added243/627
added244/627
added245/627
added246/627
added247/627
added248/627
added249/627
added250/627
added251/627
added252/627
added253/627
added254/627
added255/627
added256/627
added257/627
added258/627
added259/627
added260/627
added261/627
added262/627
added263/627
added264/627
added265/627
added266/627
added267/627
added268/627
added269/627
added270/627
added271/627
added272/627
added273/627
added274/627
added275/627
added276/627
added277/627
added278/627
added279/627
added280/627
added281/627
added282/627
added283/627
added284/627
added285/627
added286/627
added287/627
added288/627
added289/627
added290/627
added291/627
added292/627
added293/627
added294/627
added295/627
added296/627
added297/627
added298/627
added299/627
added300/627
added301/627

added338/627
added339/627
added340/627
added341/627
added342/627
added343/627
added344/627
added345/627
added346/627
added347/627
added348/627
added349/627
added350/627
added351/627
added352/627
added353/627
added354/627
added355/627
added356/627
added357/627
added358/627
added359/627
added360/627
added361/627
added362/627
added363/627
added364/627
added365/627
added366/627
added367/627
added368/627
added369/627
added370/627
added371/627
added372/627
added373/627
added374/627
added375/627
added376/627
added377/627
added378/627
added379/627
added380/627
added381/627
added382/627
added383/627
added384/627
added385/627
added386/627
added387/627
added388/627
added389/627
added390/627
added391/627
added392/627
added393/627
added394/627
added395/627
added396/627
added397/627
added398/627
added399/627
added400/627
added401/627
added402/627
added403/627
added404/627
added405/627
added406/627
added407/627
added408/627
added409/627
added410/627
added411/627
added412/627
added413/627
added414/627

added388/627
added389/627
added390/627
added391/627
added392/627
added393/627
added394/627
added395/627
added396/627
added397/627
added398/627
added399/627
added400/627
added401/627
added402/627
added403/627
added404/627
added405/627
added406/627
added407/627
added408/627
added409/627
added410/627
added411/627
added412/627
added413/627
added414/627
added415/627
added416/627
added417/627
added418/627
added419/627
added420/627
added421/627
added422/627
added423/627
added424/627
added425/627
added426/627
added427/627
added428/627
added429/627
added430/627
added431/627
added432/627
added433/627
added434/627
added435/627
added436/627
added437/627
added438/627
added439/627
added440/627
added441/627
added442/627
added443/627
added444/627
added445/627
added446/627
added447/627
added448/627
added449/627
added450/627
added451/627
added452/627
added453/627
added454/627
added455/627
added456/627
added457/627
added458/627
added459/627
added460/627
added461/627
added462/627
added463/627
added464/627

added485/627
added486/627
added487/627
added488/627
added489/627
added490/627
added491/627
added492/627
added493/627
added494/627
added495/627
added496/627
added497/627
added498/627
added499/627
added500/627
added501/627
added502/627
added503/627
added504/627
added505/627
added506/627
added507/627
added508/627
added509/627
added510/627
added511/627
added512/627
added513/627
added514/627
added515/627
added516/627
added517/627
added518/627
added519/627
added520/627
added521/627
added522/627
added523/627
added524/627
added525/627
added526/627
added527/627
added528/627
added529/627
added530/627
added531/627
added532/627
added533/627
added534/627
added535/627
added536/627
added537/627
added538/627
added539/627
added540/627
added541/627
added542/627
added543/627
added544/627
added545/627
added546/627
added547/627
added548/627
added549/627
added550/627
added551/627
added552/627
added553/627
added554/627
added555/627
added556/627
added557/627
added558/627
added559/627
added560/627
added561/627

added624/627
added625/627
added626/627
added627/627
checked
checking n=20, r=-3
added1/627
added2/627
added3/627
added4/627
added5/627
added6/627
added7/627
added8/627
added9/627
added10/627
added11/627
added12/627
added13/627
added14/627
added15/627
added16/627
added17/627
added18/627
added19/627
added20/627
added21/627
added22/627
added23/627
added24/627
added25/627
added26/627
added27/627
added28/627
added29/627
added30/627
added31/627
added32/627
added33/627
added34/627
added35/627
added36/627
added37/627
added38/627
added39/627
added40/627
added41/627
added42/627
added43/627
added44/627
added45/627
added46/627
added47/627
added48/627
added49/627
added50/627
added51/627
added52/627
added53/627
added54/627
added55/627
added56/627
added57/627
added58/627
added59/627
added60/627
added61/627
added62/627
added63/627
added64/627
added65/627
added66/627
added67/627
added68/627
added69/627
added70/627
added71/627
added72/627
added73/627
added74/627
added75/627
added76/627
added77/627
added